### Some how-tos
For rviz open:roscd hrr_common/
❯ cd rviz
❯ rviz -d hrr_cobot_hw.rvizplot 

forces:
rqt_plot /hrr_cobot/ft_sensor/wrench/force/x:y:z

In [1]:
import numpy as np
import sys
import hrr_common.utils as cu
from hrr_cobot_robot.gripper_interfaces.wsg_50_hw_local import gripper_command_handle as gch
import hrr_common as hrr_cm
import hrr_cobot_robot as hrr_rob
import spatialmath as sm
import rospy
from tqdm.notebook import tqdm, trange
from hrr_msgs.msg import PlannerAction , PlannerGoal

# ROS
import actionlib

### ros setup for the lab

In [2]:
import socket
print("for a local setup, replace the above by:\n\t" + \
      r"%env ROS_MASTER_URI=http://localhost:11311")
%env ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
print("in case this notebook does not work please check that IP below is identical:\n\t"+
      r"%env " + f"ROS_IP={ socket.gethostbyname(socket. gethostname())}\t\t<=>",end="\t")
%env ROS_IP=129.187.147.174

for a local setup, replace the above by:
	%env ROS_MASTER_URI=http://localhost:11311
env: ROS_MASTER_URI=http://hrrN3511rt2004.lsr.ei.tum.de:11311
in case this notebook does not work please check that IP below is identical:
	%env ROS_IP=129.187.147.174		<=>	env: ROS_IP=129.187.147.174


In [3]:
rospy.init_node('simple_grasping_dev')

## Gripper initialization
Assume cobot and gripper have been launched according to 
```bash
roslaunch hrr_cobot_robot hrr_cobot_hw.launch tool_name:="wsg_50"     
```

In [4]:
gripper = gch.GripperCommandHandle('/hrr_cobot/gripper')

In [5]:
gripper.reset()

## Gripper test commands (not part of grasping routine)

#Get current width
gripper.width #in m
realwidth = gripper.width-0.004 #white 3d-printed fingers are approx. 4mm wide
print(gripper.width, realwidth)

#Get current forces: [motor, finger1, finger2]
motorforce = gripper.forces[0]
motorforce

#Set desired position of gripper
width = 62 #in mm
speed = 100 #in mm/s
gripper.send_pos(width, speed)

#Set velocity (No need probably) What does it do?
gripper.set_vel(5)

#Reset gripper (Homing and rospy sleep for dt=0.1 default)
gripper.reset()

!rostopic list

# Actual grasping routine

Use cases:
Emergency lamp: Remove and dispose battery under middle cover


## Init robot control connection

In [6]:
hrr_rob.load_default_parameters("/hrr_cobot")
cobot = hrr_rob.HrrCobotControl.from_ros()
rospy.set_param("~joint_state_topic_name", "/hrr_cobot/joint_states")
rospy.set_param("~tp5_topic_name", "/hrr_cobot/comau_robot_state_controller/comau_cartesian_data")
comau_status = hrr_rob.ComauRobotState.from_ros()
rospy.sleep(1e-1)
print(comau_status)

[ERROR] [1642087538.182192]: could not set tool-frame @ /simple_grasping_dev due to missing ROS-parameter: '~tool_name'


[INFO] [1642087538.729342]: Updated force  gains: K_p:=[0.0001, 0.0001, 0.0001]
[INFO] [1642087538.732914]: Updated torque gains: K_p:=[0.0005, 0.0005, 0.0005]
q:		[ 38.09431117  60.08643771 -61.2741598   -0.68832373  58.56317576
  82.6286233 ][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[ 0.57217682 -0.44751016  0.3113642 ][m]
quat:		0.00478 <[-0.37602266  0.92659607  0.00195538]>
tool-pos:	[ 0.57217682 -0.44751016  0.3113642 ][m]
tool-rpy:	[ 135.82278442    0.42329493 -179.58639526][°]
robot-status:	ready


[WARN] [1642087538.963015]: Could not process inbound connection: Client [/hrr_cobot] wants topic [/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd] to have datatype/md5sum [hrr_msgs/HybridForceVelocityCmdStamped/6bfe9d92f9569269ce7086e736ecd0c2], but our version has [hrr_msgs/HybridForceVelocityCmdStamped/c7d32c882fe005053aba472f934842bd] Dropping connection.{'callerid': '/hrr_cobot', 'md5sum': '6bfe9d92f9569269ce7086e736ecd0c2', 'tcp_nodelay': '0', 'topic': '/hrr_cobot/sensor_track_compliant_controller/hybrid_ctrl_cmd', 'type': 'hrr_msgs/HybridForceVelocityCmdStamped'}


## Move to calibration pose to start (not necessary)

In [ ]:
cobot.move_to_pose(cobot.FK(cobot.q_calib))
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

# Define parameters

## Parameters given from outside (task planner)
Set to dummy values here

In [ ]:
from geometry_msgs.msg import PoseStamped #For testing purposes
object_center = PoseStamped() #PoseStamped
dispose_off_location_pose = PoseStamped() #PoseStamped
gripper_open_width = 0 #float32 #mm
gripper_close_width = 0 #float32 #mm


### Compute goal_pose of the robot (grasping pose) from object center
goal_pose: the pose of the robot EE (SE3) when initially grasping the object, i.e. during closing of the gripper

This computation needs to consider:
* translation from object center to grasping center point (might be zero)
* translation from grasping center point to robot EE (depending on length of gripper & adapters)
* orientation of the robot EE: grasp from above, i.e. orientation with EE pointing down
* last link rotated such that gripper aligns with object as wanted (compute from object orientation)

In [ ]:
#goal_pose = cu.calc_EE_goal_pose(nvec, object_location, C_p_CE=disp_tip_EE)
goal_pose = sm.SE3()

### Compute dispose_pose of the robot EE (where to stop grasping)
dispose_pose: the pose of the robot EE (SE3) when dropping the object

probably already given from outside in dispose_off_location_pose, unless orientation is empty?

In [ ]:
dispose_pose = sm.SE3()

### "Ensure" gripper open and close width are in mm

In [7]:
if gripper_open_width < 1:
    #probably given in meters
    open_width = gripper_open_width*1000
else:
    open_width = gripper_open-width
if gripper_close_width < 1:
    #probably given in meters
    open_width = gripper_close_width*1000
else:
    open_width = gripper_close_width    

NameError: name 'gripper_open_width' is not defined

### For demos, we set outside parameters manually

In [177]:
#cobot.update_tf()
#np.save('goal_pose_simgrasp',cobot.T_B_E_robot.A,allow_pickle=True) #saves current pose as goal_pose
goal_pose = np.load('goal_pose_simgrasp.npy')
goal_pose

array([[-0.71841924, -0.69554824,  0.00929714,  0.56683789],
       [-0.69553229,  0.71847469,  0.00538119, -0.45039294],
       [-0.01042264, -0.00260051, -0.9999423 ,  0.27102234],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
dispose_pose = np.load('dispose_pose_vac.npy')
dispose_pose = sm.SE3(np.array([0,0,0.07]))@sm.SE3(dispose_pose) #for this task go a bit higher with the dispose pose
sm.SE3(dispose_pose) #prob go down 10-20cm

  -0.6713   -0.7408    0.02192   0.5709    
  -0.7408    0.6716    0.01315   0.3754    
  -0.02447  -0.007406 -0.9997    0.505     
   0         0         0         1         


In [10]:
#np.save('final_pose_vac',final_pose.A,allow_pickle=True)
final_pose = np.load('final_pose_vac.npy')
final_pose = sm.SE3(np.array([0,0,0.04]))@sm.SE3(final_pose)
sm.SE3(final_pose)

  -0.6713   -0.7408    0.02192   0.5709    
  -0.7408    0.6716    0.01315   0.3754    
  -0.02447  -0.007406 -0.9997    0.325     
   0         0         0         1         


In [36]:
#Width of gripper during grasp
close_width = 78 # in mm! Note that the actual width of the grasp is approx. 4mm smaller, b/c of finger thickness
#Width of gripper before grasp
open_width = 100 # in mm! Note that the actual width of the grasp is approx. 4mm smaller, b/c of finger thickness

## Hardcoded parameters and computation of Pre-pose

In [12]:
#Normal vector from goal_pose to pre-pose
nvec_prepose = np.array([0, 0, 1]) #[0 0 1] means pre-pose is above goal-pose, i.e. we approach from above when grasping

#Defaul closing/opening speed of gripper in mm/s
default_speed = 100

#desired distance from goal-pose to pre-pose
pre_pose_dist = 0.06

#Thickness of fingers used to adjust gripper.width (in m)
finger_thickness = 0.004

### Util methods with parameters included

In [13]:
def getGripWidth():
    return 1000*(gripper.width - finger_thickness)
getGripWidth()

105.23503339290619

In [14]:
def setGripWidth(obj_width):
    gripper.send_pos(obj_width+finger_thickness, default_speed)

In [53]:
?gripper.send_pos

### Compute pre-pose from goal-pose

In [111]:
#Compute pre-pose from nvec, location, pre_pose_dist, disp_tip_EE
nvec = cu.normalize_vec(nvec_prepose) #Make sure nvec is normalized
pre_pose = sm.SE3(nvec* pre_pose_dist) @ sm.SE3(goal_pose)
print(pre_pose,sm.SE3(goal_pose))

  -0.7178   -0.6962    0.008319  0.5664    
  -0.6962    0.7178    0.006333 -0.4509    
  -0.01038  -0.001246 -0.9999    0.3441    
   0         0         0         1         
   -0.7178   -0.6962    0.008319  0.5664    
  -0.6962    0.7178    0.006333 -0.4509    
  -0.01038  -0.001246 -0.9999    0.2841    
   0         0         0         1         



### Use this to manually check poses

In [40]:
cobot.move_to_pose(pre_pose, err_gain = None, v_max = 0.005) #pre_pose, cobot.FK(cobot.q_calib)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642087924.614089]: reached goal pose at step 532 / 10000


### Init traj planner

In [58]:
 # Initialize the planner action client.
    
# >> Remeber to run the planner node (rosrun planner planner_server_v2.py) in terminal
# >> 
#rospy.init_node('planner_client')

client1 = actionlib.SimpleActionClient('planner', PlannerAction)
client1.wait_for_server()

# Sending deride end configuration
goal = PlannerGoal()
goal.goal_pose.header.frame_id = "map"
goal.goal_pose.header.stamp = rospy.Time.now()

# Robot movement commands start here

### Start at calibration pose

In [142]:
cobot.move_to_pose( cobot.FK(cobot.q_calib), err_gain = None, v_max = 0.03) #start_pose, cobot.FK(cobot.q_calib)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642089709.575976]: reached goal pose at step 1007 / 10000


### Go to pre-pose using traj planner

In [143]:
gripper.reset()

In [144]:
pre_pose_quat = cu.homog2quat(pre_pose)
pre_pose_pos = cu.homog2pos(pre_pose)
#position of end_configuration
goal.goal_pose.pose.position.x = pre_pose_pos[0]
goal.goal_pose.pose.position.y = pre_pose_pos[1]
goal.goal_pose.pose.position.z = pre_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = pre_pose_quat.x
goal.goal_pose.pose.orientation.y = pre_pose_quat.y
goal.goal_pose.pose.orientation.z = pre_pose_quat.z
goal.goal_pose.pose.orientation.w = pre_pose_quat.w

In [145]:
 client1.send_goal(goal)

### set gripper to open_width

In [174]:
setGripWidth(open_width)

### Approach and grasp (set gripper to close_width)

In [175]:
cobot.move_to_pose(goal_pose)

In [176]:
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642089833.794976]: reached goal pose at step 246 / 10000


In [178]:
setGripWidth(close_width) #78

### Move back to pre-pose

In [179]:
cobot.move_to_pose(pre_pose)

In [180]:
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()

  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] [1642089899.638121]: reached goal pose at step 251 / 10000


### Check if object is grasped


In [155]:
print(gripper.forces[0])

0.0


### move to dispose location using traj planner

In [ ]:
dispose_pose_quat = cu.homog2quat(dispose_pose)
dispose_pose_pos = cu.homog2pos(dispose_pose)

#position of end_configuration
goal.goal_pose.pose.position.x = dispose_pose_pos[0]
goal.goal_pose.pose.position.y = dispose_pose_pos[1]
goal.goal_pose.pose.position.z = dispose_pose_pos[2]

#orientation of end_configuration
goal.goal_pose.pose.orientation.x = dispose_pose_quat.x
goal.goal_pose.pose.orientation.y = dispose_pose_quat.y
goal.goal_pose.pose.orientation.z = dispose_pose_quat.z
goal.goal_pose.pose.orientation.w = dispose_pose_quat.w

In [ ]:
 client1.send_goal(goal)

### release object

In [ ]:
cobot.move_to_pose(final_pose,err_gain = None, v_max = 0.03)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()
gripper.reset()

In [ ]:
cobot.move_to_pose( cobot.FK(cobot.q_calib), err_gain = None, v_max = 0.03) #start_pose, cobot.FK(cobot.q_calib)
T = int(100 * cobot.hz)
for t in trange(T):
    if cobot.state is None:
        rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
        break
    elif cobot.state == "error":
        rospy.logerror(f"robot in ERROR state")
    cobot.update()